In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\HP\\Desktop\\ML\\Customer_Churn_Predictor'

In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/kalpesh2003patel/Customer_Churn_Predictor.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="kalpesh2003patel"
os.environ["MLFLOW_TRACKING_PASSWORD"]="247a5e8972461026c26da95f46a42599bc30ae6c"

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [5]:
from ChurnPredictor.constants import *
from ChurnPredictor.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.CatBoostClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/kalpesh2003patel/Customer_Churn_Predictor.mlflow",
           
        )

        return model_evaluation_config


In [7]:
import os
from ChurnPredictor import logger
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        accuracy = accuracy_score(actual, pred)
        return accuracy
    

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            logger.info(predicted_qualities)
            logger.info(test_y)


            accuracy = self.eval_metrics(test_y, predicted_qualities)
            
            scores = {"accuracy": accuracy}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="CatboostClassifierModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-05-27 17:37:38,672: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-27 17:37:38,675: INFO: common: yaml file: params.yaml loaded successfully]


[2024-05-27 17:37:38,680: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-27 17:37:38,682: INFO: common: created directory at: artifacts]
[2024-05-27 17:37:38,684: INFO: common: created directory at: artifacts/model_evaluation]
[2024-05-27 17:38:15,800: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /kalpesh2003patel/Customer_Churn_Predictor.mlflow/api/2.0/mlflow/runs/create]
[2024-05-27 17:38:16,574: INFO: 4151194231: [0 0 0 ... 1 1 0]]
[2024-05-27 17:38:16,576: INFO: 4151194231:       Churn
0         0
1         0
2         0
3         1
4         0
...     ...
2105      0
2106      0
2107      1
2108      0
2109      0

[2110 rows x 1 columns]]
[2024-05-27 17:38:16,590: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'CatboostClassifierModel' already exists. Creating a new version of this model...
2024/05/27 17:38:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: CatboostClassifierModel, version 3
Created version '3' of model 'CatboostClassifierModel'.
